# 함수 데커레이터와 클로저

함수 데커레이터는 소스 코드에 있는 함수를 '표시'해서 함수의 작동을 개선할 수 있게 해준다.
자기만의 데커레이터를 구현하고자 한다면 클로저를 속속들이 이해해야 하며, 그러고 나면 nonlocal이 필요해진다.

데커레이터에서 사용하는 것 외에도, 클로저는 콜백을 이용한 효율적인 비동기 프로그래밍과 필요에 따라 함수형 스타일로 코딩하는 데에도 필수적이다.
- 파이썬이 데커레이터 구문을 평가하는 방식
- 변수가 지역 변수인지 파이썬이 판단하는 방식
- 클로저의 존재 이유와 작동 방식
- nonlocal로 해결할 수 있는 문제
이런 기반을 갖추고 나면 다음과 같이 데커레이터 주제를 심도 있게 다룰 수 있다.
- 잘 작동하는 데커레이터 구현하기
- 표준 라이브러리에서 제공하는 재미있는 데커레이터들
- 매개변수화된 데커레이터 구현하기

## 데커레이터 기본 지식
데커레이터는 다른 함수를 인수로 받는 콜러블(데커레이트된 함수)이다.
데커레이터는 데커레이트된 함수에 어떤 처리를 수행하고, 함수를 반환하거나 함수를 다른 함수나 콜러블 객체로 대체한다.

In [3]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

target()
target

running inner()


<function __main__.deco.<locals>.inner()>

엄밀히 말해 데커레이터는 편리 구문일 뿐이다. 데커레이터는 다른 함수를 인수로 전달해서 호출하는 일반적인 콜러블과 동일하다.
그렇지만 런테임에 프로그램 행위를 변경하는 메타프로그래밍을 할 때 데커레이터가 상당히 편리하다.

## 파이썬이 데커레이터를 실행하는 시점
데커레이터의 핵심 특징은 데커레이트된 함수가 정의된 직후에 실행된다는 것이다. 이는 일반적으로 파이썬이 모듈을 로딩하는 시점, 즉 임포트 타임에 실행된다.

데커레이터는 모듈 내의 다른 어떠한 함수보다 먼저 실행된다.(main보다)
이를 통해 임포트 타임과 런타임이라고 부르는 것의 차이를 보여준다.
- 데커레이터 함수가 데커레이트되는 함수와 같은 모듈에 정의되어 있다. 일반적으로 실제 코드에서는 데커레이터를 정의하는 모듈과 데커레이터를 적용하는 모듈을 분리해서 구현한다.
- register() 데커레이터가 인수로 전달된 함수와 동일한 함수를 반환한다. 실제 코드에서 대부분의 데커레이터는 내부 함수를 정의해서 반환한다.

## 데커레이터로 개선한 전략 패턴


In [ ]:
promos = []
def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구매하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    """최대로 할인받을 금액을 반환한다.
    """
    return max(promo(order) for promo in promos)

이 방법은 다음과 같은 장점이 있다.
- 프로모션 전략 함수명이 특별한 형태로 되어 있을 필요 없다(이제는 함수명이 _promo로 끝나지 않아도 된다).
- @promotion 데커레이터는 데커레이트된 함수의 목적을 명확히 알려주며, 임시로 어떤 프로모션을 배제할 수 있다. 단지 데커레이터만 주석처리하면 된다.
- 프로모션 할인 전략을 구현한 함수는 `@promotion` 데커레이터가 적용되는 한 어느 모듈에서는 정의할 수 있다.

## 클로저
클로저는 함수 본체에서 정의하지 않고 참조하는 비전역 변수를 포함한 확장 범위를 가진 함수다.
함수 본체 외부에 정의된 비전역 변수에 접근할 수 있다는 것이 중요하다.

In [ ]:
# 전체 이력을 유지하지 않고 이동 평균 계산하기(nonlocal)
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

## 간단한 데커레이터 구현하기


In [6]:
# 함수의 실행시간을 출력하는 데커레이터

import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [7]:
# clock 데커레이터 사용하기
import time

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! = ', factorial(6))

**************************************** Calling snooze(.123)
[0.12658046s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000121s] factorial(1) -> 1
[0.00001737s] factorial(2) -> 2
[0.00002742s] factorial(3) -> 6
[0.00003725s] factorial(4) -> 24
[0.00004775s] factorial(5) -> 120
[0.00005983s] factorial(6) -> 720
6! =  720
